## Vers le Retrieval Augmented Generation
---

### Librement adapté des supports de Rémy DECOUPES

---

**Objectif** : 
1. Mise en place d'un système RAG (Retrieval Augmented Generation)
2. Réaliser un topic modelling

On repart des bases du TP2: vous devez savoir faire tourner ollama sur votre machine

In [ ]:
# !pip install langchain langchain-community langchain_ollama

In [ ]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.2") # 'gemma3:270m' pour aller plus vite

# Envoi d’un prompt
response = llm.invoke("Peux-tu lister les maladies des plantes à La Réunion ?")

print(response)

## 1. Mise en place du RAG : Retrieval Augmented Generation


In [ ]:
# ATTENTION: les imports changent tout le temps... => StackOverFlow
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

### 1.1 Mise en place du corpus

Utilisation du jeu de données : https://huggingface.co/datasets/KisanVaani/agriculture-qa-english-only

In [ ]:
from datasets import load_dataset
import pandas as pd

ds = load_dataset("KisanVaani/agriculture-qa-english-only", split="train")

df = pd.DataFrame(ds)
df

In [ ]:
corpus = df["answers"].to_list()
corpus[:10]

### 1.2 Calculer les embeddings des documents

In [ ]:
from langchain.docstore.document import Document

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

documents = [Document(page_content=text) for text in corpus[:200]]

docs = text_splitter.split_documents(documents)

In [ ]:
modelPath = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

In [ ]:
vectorstore = FAISS.from_documents(docs, embeddings)

In [ ]:
question = "which diseases can be spread by flies?"
searchDocs = vectorstore.similarity_search(question)
print(searchDocs[0].page_content)

### 1.3 Préparer son LLM

In [ ]:
from langchain.chains import RetrievalQA

retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) 

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",        # méthode pour combiner les docs
    retriever=retriever,
    return_source_documents=True
)

<img align="right" src="https://upload.wikimedia.org/wikipedia/commons/5/55/Question_Mark.svg">

**Questions** :

---
- Quel est l'argument k = 3 ?

In [ ]:
query = "which diseases can be spread by flies?"
result = rag_chain({"query": query})

print("Réponse générée :\n", result["result"])
print("\nDocuments utilisés :\n", result["source_documents"])

## 2. Adapter votre système RAG

<img align="right" src="https://upload.wikimedia.org/wikipedia/commons/5/55/Question_Mark.svg">

**Faîtes varier** :

--- 

- Le nombre de documents utilisés par le contexte
- Changer de jeu de données
- Changer de modèle LLM
- Changer de modèle d'embedding

In [ ]:
# vos tests

## 3. Topic Modelling

L'objectif est d'identifier des sujets dans ce corpus de titre.
Nous allons utiliser `BERTopic`.

![img](https://maartengr.github.io/BERTopic/algorithm/default.svg)


Ce paquet utilise donc tout ce qu'on a appris avec une configuration particulière de TF-IDF :

![img2](https://maartengr.github.io/BERTopic/algorithm/c-TF-IDF.svg)

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic()

topics, probs = topic_model.fit_transform(corpus[:200])

In [ ]:
topic_model.get_topic_info()

In [ ]:
topic_model.get_topic(0)

Nous voyons qu'il y a beaucoup de `stopwords` !

Il est possible de tout configurer avec Bertopic.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words="english")
topic_model = BERTopic(vectorizer_model=vectorizer_model)

topics, probs = topic_model.fit_transform(corpus[:200])

topic_model.get_topic_info()

In [ ]:
topic_model.get_topic_info()